https://beta.bls.gov/dataQuery/find?fq=survey:[ln]&s=popularity:D

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
state_number = list(range(1,57))
# Series_ID = ["LASST060000000000006"]
Series_ID = [];
for i in state_number:
    if i < 10:
        Series_ID.append("LASST0"+str(i)+"0000000000006")
    else:    
        Series_ID.append("LASST"+str(i)+"0000000000006")
    # last number indicates different data types, 3,4,5,6,7 is available

SaveFileAs = "Labor Force Data.csv"

In [3]:
#Get your jsonified api data here

#Collecting only 2017 Data
startyear = 2017
endyear = 2017

base_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    
headers = {'Content-type': 'application/json'}

State = []
Labor_Force = []

for this_data in Series_ID:
    #Collect data for this series ID
    data = json.dumps({"seriesid": [this_data],"startyear":startyear, "endyear":endyear,"registrationkey":"a1996666110d420cbf38f706289109f6"})
    # Request for api data
    Complete_Data = requests.post(f"{base_URL}",data=data, headers=headers)
    # jsonify the data
    Complete_Data = Complete_Data.json()
    
    try:
        this_Labor_Force= Complete_Data["Results"]["series"][0]["data"][0]["value"]
        Labor_Force.append(this_Labor_Force)
        
        #Beautiful Soup to get state of data
        website = "https://beta.bls.gov/dataViewer/view/timeseries/"
        url = website+this_data;

        response = requests.get(url)
        soup = bs(response.text, "html.parser")

        table_text = soup.find_all("table",{"id": "catalogDataTable1"})
        this_state = table_text[0].find("tr").find_all("td")[2].text
        this_state = this_state.replace("Labor Force: ","").replace(" (S)","")
        State.append(this_state)
        
    except(IndexError):
        print(IndexError)
    
    #Beautiful Soup to get state of data
#     website = "https://beta.bls.gov/dataViewer/view/timeseries/"
#     url = website+this_data[0];

#     response = requests.get(url)
#     soup = bs(response.text, "html.parser")

#     table_text = soup.find_all("table",{"id": "catalogDataTable1"})
#     try:
#         this_state = table_text[0].find("tr").find_all("td")[2].text
#         this_state = this_state.replace("Labor Force: ","").replace(" (S)","")
#         State.append(this_state)

#     except(IndexError):
#         State.append("NA")
    

    

# for loop to iterate through the api data and grab all the important information
# for i in Complete_Data["Results"]["series"][0]["data"]:
#     Years.append(i["year"])
#     Months.append(int(i["period"].replace('M', '')))
#     Unemployment_Rate.append(i["value"])

# Complete_Data



<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>


In [4]:
Labor_Force_db = pd.DataFrame({"State":State,
                              "Labor Force":Labor_Force});

In [7]:
Labor_Force_db.to_csv(f"../data/{SaveFileAs}",index=False)